## Train Instrument Identification Model

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Load Training CSV File

Displays dataframe for verification purposes.

In [2]:
# Load the training CSV file
csv_file = '../../image_labels/attitude_labels/attitude_training_1.csv'
training_df = pd.read_csv(csv_file)

image_directory = '../../media/attitude_training_1/'
training_df['image_path'] = training_df['image_path'].apply(lambda x: os.path.join(image_directory, x))

training_df

,image_path,attitude
0,../../media/attitude_training_1/attitude_train...,0.0
1,../../media/attitude_training_1/attitude_train...,0.0
2,../../media/attitude_training_1/attitude_train...,0.0
3,../../media/attitude_training_1/attitude_train...,0.0
4,../../media/attitude_training_1/attitude_train...,0.0
...,...,...
17377,../../media/attitude_training_1/attitude_train...,0.5
17378,../../media/attitude_training_1/attitude_train...,0.5
17379,../../media/attitude_training_1/attitude_train...,0.5
17380,../../media/attitude_training_1/attitude_train...,0.5


## Helper Function - Load Image

In [3]:
def load_image(file_path):
    image = cv2.imread(file_path)

    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

## Load Images and apply labels from CSV

In [4]:
# Load images into a NumPy array
images = np.array([load_image(file_path) for file_path in training_df['image_path']])

target_column_name = 'attitude'

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_df[target_column_name])

# Optionally, one-hot encode the labels (if using categorical cross-entropy)
num_classes = len(label_encoder.classes_)
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes)


# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)

## Execute - Run Training Model

In [5]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 205, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Load and preprocess your dataset
# You can use TensorFlow's `ImageDataGenerator` for data augmentation and preprocessing.

# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

# Save the trained model
model.save('attitude_final_5_epoch.h5')

Epoch 1/5
435/435 [==============================] - 98s 225ms/step - loss: 1.6452 - accuracy: 0.5356 - val_loss: 0.6048 - val_accuracy: 0.8145
Epoch 2/5
435/435 [==============================] - 83s 191ms/step - loss: 0.4194 - accuracy: 0.8637 - val_loss: 0.4042 - val_accuracy: 0.8729
Epoch 3/5
435/435 [==============================] - 84s 192ms/step - loss: 0.2657 - accuracy: 0.9100 - val_loss: 0.3830 - val_accuracy: 0.8766
Epoch 4/5
435/435 [==============================] - 84s 193ms/step - loss: 0.2010 - accuracy: 0.9289 - val_loss: 0.3679 - val_accuracy: 0.8867
Epoch 5/5
435/435 [==============================] - 85s 195ms/step - loss: 0.1721 - accuracy: 0.9376 - val_loss: 0.3169 - val_accuracy: 0.9005


/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
